# Validate EIA-930 data against net generation outputs

In [76]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
from datetime import datetime
from datetime import timedelta
import json

import requests

In [71]:
year = 2020

In [72]:
# EIA-930 data after timestamp adjustments but no cleaning
raw = pd.read_csv("../data/outputs/2020/eia930/eia930_raw.csv", index_col=0, parse_dates=True)

In [73]:
GEN_ID = "EBA.{}-ALL.NG.H"
path = f"../data/results/{year}/power_sector_data/hourly/us_units/"
cors = {}
percent_difs = {}
annual_gen = {}
for ba_f in os.listdir(path):
    ba = ba_f.replace(".csv", "")
    print(ba, end="...")
    col_name = GEN_ID.format(ba)
    if col_name not in raw.columns: 
        continue
    else:
        dat = pd.read_csv(path+ba_f, parse_dates=["datetime_utc"])
        dat = dat[dat.fuel_category=="total"]
        dat = dat.merge(raw[ col_name], left_on="datetime_utc", right_index=True)
        c = dat[["net_generation_mwh", col_name]].corr().to_numpy()[0,1]
        cors[ba] = c
        difs = (dat[col_name]-dat["net_generation_mwh"])/dat["net_generation_mwh"]
        difs = difs.replace(np.inf, np.nan)
        percent_difs[ba] = difs.median()
        annual_gen[ba] = dat["net_generation_mwh"].sum()

OHMS...FMPP...ERCO...TPWR...AMPL...SOCO...IPCO...PJM...WWA...EEI...AZPS...TEC...DOPD...YAD...IID...HGMA...DEAA...CPLW...SPA...GVL...ORMS...FPL...TAL...SEC...JEA...GCPD...TVA...TIDC...HECO...SCEG...INMS...CPLE...SEPA...AKMS...CHPD...PNM...WAUW...WACM...WALC...NWMT...PACE...AVA...SC...NSB...GWA...HIMS...LGEE...TEPC...ISNE...SRP...GRIS...HST...LDWP...RIMS...FPC...PACW...GRIF...PSEI...AECI...CSTO...DUK...AVRN...CEA...MISO...AEC...PSCO...OVEC...SWPP...BANC...NYIS...EPE...NBSO...SCL...BPAT...NEVP...CISO...PGE...

In [74]:
out = pd.DataFrame(data={"Difference as percent of hourly-egrid":percent_difs, "Correlation":cors, "Annual BA generation":annual_gen})
out = out.sort_values("Annual BA generation", ascending=False)
out.to_csv(f"../data/results/{year}/validation_metrics/us_units/compare_930_hourlyegrid.csv")

# Visualize BA of interest

In [75]:
ba = "BPAT"
col_name = GEN_ID.format(ba)
dat = pd.read_csv(path+ba+".csv", parse_dates=["datetime_utc"])
dat = dat[dat.fuel_category=="total"]
dat = dat.merge(raw[ col_name], left_on="datetime_utc", right_index=True)

px.line(dat, x="datetime_utc", y=["net_generation_mwh", col_name])

# ~~Validate against carbon intensity and emissions data~~

Scraping data is too slow (1 hour per BA) and users without a paid API plan won't have enough requests. Instead, re-calculate rates from EIA-930 using Singularity per-fuel emission factors (derived from eGRID) 

* Scrape data from Singularity if not already downloaded
* Compare

Data location: 

`data/downloads/singularity/\<year\>_generated_rate.csv`

`data/downloads/singularity/\<year\>_consumed_rate.csv`

To scrape data, env var SINGULARITY_API_KEY should be set 

In [2]:
year = 2020
ba = "ISNE"


In [25]:
EIA_REGIONS = { # regions for which singularity name is prefixed by EIA.
    'BPAT',
    'CISO',
    'ISNE',
    'MISO',
    'NYIS',
    'PJM',
    'SWPP',
}

def download_singularity_carbon_intensities(ba, year):
    # TODO GAILIN REMOVE API KEY

    os.makedirs(f"../data/downloads/singularity/{year}", exist_ok=True)
    fpath = f"../data/downloads/singularity/{year}/{ba}_rate.csv"
    if not os.path.exists(fpath): 
        url = "https://api.singularity.energy/v1/region_events/search"
        headers = {
            'X-Api-Key': os.environ['SINGULARITY_API_KEY'],
        }
        params = {
            'event_type': "carbon_intensity",
            'region': ("EIA." + ba) if ba in EIA_REGIONS else ba
        }

        dat = {}

        # Iterate through weeks in year 
        start = datetime(year=year, month=1, day=1, hour=0, minute=0)
        while start.year == year:
            end = start + timedelta(weeks=1) 
            # Format params for search 
            params["start"] = start.strftime("%Y-%m-%dT%H:%MZ")
            params["end"] = end.strftime("%Y-%m-%dT%H:%MZ")
            # Show some progress 
            #print(start.strftime("%Y-%m-%d"), end="...")
            # Update start 
            start = end 

            next = 1
            last = 1

            # Get all pages of data for this week
            while next <= last: 
                params['page'] = next
                response = requests.request("GET", url, headers=headers, params=params)
                if len(response.json()['data']) == 0:
                    print(f"Found no data for {params['region']}, will not create file")
                    return
                if "next" in response.json()['meta']['pagination']: 
                    next = response.json()['meta']['pagination']['next']
                else: 
                    next = next + 1
                last = response.json()['meta']['pagination']['last']
                for d in response.json()['data']:
                    # Only bother if this is real data, not infilled five minute intervals 
                    if d['start_date'][-11:-9] != "00":
                        continue
                    # Only bother if eGRID source year is 2019: some data is computed using 2018 factors
                    try:
                        if "2019" not in d['meta']['generated_emissions_source']:
                            continue
                    except:
                        print(f"Failed with response {d}")
                        raise Exception

                    # Parse dat to consistent names based on source 
                    this_dat = dat.get(d['start_date'], {})
                    try:
                        for accounting_type in ["consumed","generated"]:
                            if f"{accounting_type}_rate" not in d["data"].keys():
                                continue
                            if "eq" in d["meta"]["generated_emissions_source"]: # CO2 equivalent
                                this_dat[f"{accounting_type}_eq"] = d["data"][f"{accounting_type}_rate"]
                            elif "u" in d["meta"]["generated_emissions_source"]: # unadjusted
                                this_dat[f"{accounting_type}_u"] = d["data"][f"{accounting_type}_rate"]
                            else: # normal adjustments
                                this_dat[f"{accounting_type}"] = d["data"][f"{accounting_type}_rate"]
                        # Index by start date
                        dat[d['start_date']] = this_dat
                        
                    except:
                        print(f"Failed on \n {d} \n\n")
                        raise Exception
         
        ba_dat = pd.DataFrame(dat).transpose()
        ba_dat.index = pd.to_datetime(ba_dat.index)
        ba_dat.to_csv(fpath)
    else: 
        print(f"File {fpath} exists")




In [ ]:
download_singularity_carbon_intensities("WWA", 2020)

In [ ]:
for ba in os.listdir("../data/results/2020/power_sector_data/hourly/us_units/"):
    ba = ba.replace(".csv","")
    print(ba, end="...")
    download_singularity_carbon_intensities(ba, 2020)

In [ ]:
for ba in os.listdir("../data/results/2019/power_sector_data/hourly/us_units/"):
    ba = ba.replace(".csv","")
    print(ba)
    download_singularity_carbon_intensities(ba, 2019)

# Alternative from above: calculate real-time-rates from 930 + eGRID

In [18]:
eia930 = pd.read_csv(f"../data/outputs/{year}/eia930/eia930_raw.csv", parse_dates=True, index_col=0)

In [62]:
## Load factors from Singularity API

# Use last year's egrid because that's all we have in real time
# TODO: could expand to other pollutants if we use eGRID download 
url = f"https://api.singularity.energy/v1/emissions/" 
egrid_year = str(year-1) # use last year as eGRID year

headers = {
    'X-Api-Key': os.environ['SINGULARITY_API_KEY'],
}

factors = {}

for adjustment in ["adjusted", "unadjusted"]: 
    adjusted = adjustment == "adjusted"
    key = f"EGRID_{egrid_year}" if adjusted else f"EGRID_u{egrid_year}"
    response = requests.request("GET", url+key, headers=headers)
    factors[adjustment] = json.loads(response.content)["data"]


In [63]:
## For each BA, use singularity factors to calculate emission rate 
bas_to_calc = [ba.replace(".csv", "") for ba in os.listdir("../data/results/2020/power_sector_data/hourly/us_units/")]

fuel_categories = {
    "coal":"COL",
    "natural_gas":"NG",
    "other":"OTH",
    "hydro":"WAT",
    "wind":"WND",
    "solar":"SUN",
    "nuclear":"NUC",
    "petroleum":"OIL"
}

for ba in bas_to_calc:
    singularity_ba = "EIA." + ba if ba in EIA_REGIONS else ba
    if singularity_ba not in factors[adjustment].keys():
        print(f"missing ba {singularity_ba}")
        continue

    out = pd.DataFrame(index=eia930.index, columns=["adjusted_carbon","unajusted_carbon", "adjusted_rate", "unadjusted_rate"])

    for adjustment in ["adjusted", "unadjusted"]:
        s_fuels = list(factors[adjustment][singularity_ba].keys())
        s_factors = [factors[adjustment][singularity_ba][f]['value'] for f in s_fuels]
        fuels = [fuel_categories[f] for f in s_fuels]
        generation_labels = [f"EBA.{ba}-ALL.NG.{f}.H" for f in fuels]

        out.loc[:,f"{adjustment}_carbon"] = eia930[generation_labels].mul(s_factors, axis='columns').sum(axis='columns')
        out.loc[:,f"{adjustment}_rate"] = out.loc[:,f"{adjustment}_carbon"] / eia930.loc[:,f"EBA.{ba}-ALL.NG.H"]

    os.makedirs(f"../data/outputs/{year}/validation/real_time_rate/", exist_ok=True)
    out.to_csv(f"../data/outputs/{year}/validation/real_time_rate/{ba}.csv")


missing ba OHMS
missing ba AMPL
missing ba EEI
missing ba ORMS
missing ba JEA
missing ba HECO
missing ba INMS
missing ba AKMS
missing ba NSB
missing ba HIMS
missing ba GRIS
missing ba RIMS
missing ba CSTO
missing ba CEA
missing ba OVEC
missing ba NBSO


# Rate: correlations and percent differences

Evaluation of rates

In [77]:
gen_path = f"../data/results/{year}/power_sector_data/hourly/us_units/"
consumed_path = f"../data/results/{year}/carbon_accounting/hourly/us_units/"

In [78]:
year = 2020

In [81]:
percent_difs = {}
cors = {}
for ba in os.listdir(f"../data/outputs/{year}/validation/real_time_rate/"):
    if ba == ".DS_Store": # just some os stuff
        continue 
    ba = ba.replace(".csv", "")
    singularity_dat = pd.read_csv(f"../data/outputs/{year}/validation/real_time_rate/{ba}.csv", index_col=0, parse_dates=True)
    # hourly_consumed = pd.read_csv(consumed_path+ba+".csv",
    #     usecols=["datetime_utc", "consumed_co2_rate_lb_per_mwh_for_electricity", "consumed_co2_rate_lb_per_mwh_adjusted"], 
    #     index_col="datetime_utc", parse_dates=True)
    hourly_generated = pd.read_csv(gen_path+ba+".csv", 
        usecols=["datetime_utc", "generated_co2_rate_lb_per_mwh_for_electricity", "generated_co2_rate_lb_per_mwh_for_electricity_adjusted", "co2_mass_lb", "fuel_category"], 
        index_col="datetime_utc", parse_dates=True)
    hourly_generated = hourly_generated.loc[hourly_generated.fuel_category=="total"]
    hourly_generated = hourly_generated.sort_index()
    all_dat = pd.concat([singularity_dat, hourly_generated], axis='columns')
    all_dat = all_dat.sort_index()
    cors[ba] = all_dat[["generated_co2_rate_lb_per_mwh_for_electricity_adjusted", "adjusted_rate"]].corr().to_numpy()[0,1]
    percent_difs[ba] = ((all_dat["adjusted_rate"] - all_dat["generated_co2_rate_lb_per_mwh_for_electricity_adjusted"])/all_dat["generated_co2_rate_lb_per_mwh_for_electricity_adjusted"]).median()



In [84]:
#percent_difs

In [85]:
out = pd.DataFrame(data={"Difference as percent of OGE":percent_difs, "Correlation":cors, "Annual BA generation":annual_gen})
out = out.sort_values("Annual BA generation", ascending=False)
out.to_csv(f"../data/results/{year}/validation_metrics/us_units/compare_real_time_rates.csv")

In [86]:
out.head()

,Difference as percent of OGE,Correlation,Annual BA generation
PJM,0.004042,0.754662,7.954303e+08
MISO,-0.046745,0.977620,6.169311e+08
ERCO,-0.016253,0.988708,4.101009e+08
SWPP,-0.002570,0.991585,2.590165e+08
SOCO,-0.012536,0.983452,2.406772e+08


# Visualize emission rate differences

In [93]:
ba_of_interest = "PJM"

In [99]:
real_time = pd.read_csv(f"../data/outputs/{year}/validation/real_time_rate/{ba_of_interest}.csv", index_col=0, parse_dates=True)

In [95]:
hourly_consumed = pd.read_csv(consumed_path+ba_of_interest+".csv",
    usecols=["datetime_utc", "consumed_co2_rate_lb_per_mwh_for_electricity", "consumed_co2_rate_lb_per_mwh_for_electricity_adjusted"], 
    index_col="datetime_utc", parse_dates=True)
hourly_generated = pd.read_csv(gen_path+ba_of_interest+".csv", 
    usecols=["datetime_utc", "generated_co2_rate_lb_per_mwh_for_electricity", "generated_co2_rate_lb_per_mwh_for_electricity_adjusted", "co2_mass_lb", "fuel_category"], 
    index_col="datetime_utc", parse_dates=True)

In [100]:
all_dat = pd.concat([real_time, hourly_consumed, hourly_generated.loc[hourly_generated.fuel_category=="total"]], axis='columns')
all_dat = all_dat.sort_index()

In [102]:
real_time.head()

,adjusted_carbon,unajusted_carbon,adjusted_rate,unadjusted_rate,unadjusted_carbon
2019-06-30 12:00:00+00:00,0.0,NaN,NaN,NaN,0.0
2019-06-30 13:00:00+00:00,0.0,NaN,NaN,NaN,0.0
2019-06-30 14:00:00+00:00,0.0,NaN,NaN,NaN,0.0
2019-06-30 15:00:00+00:00,0.0,NaN,NaN,NaN,0.0
2019-06-30 16:00:00+00:00,0.0,NaN,NaN,NaN,0.0


In [103]:
px.line(all_dat, x=all_dat.index, y=["generated_co2_rate_lb_per_mwh_for_electricity", "adjusted_rate"])